In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')


In [ ]:
all_price = pd.concat([one, two, three, four, top_rest], axis =0)

In [ ]:
all_food.info()

In [ ]:
all_food

In [ ]:
top_rest['id'] = pd.to_numeric(top_rest.id)
all_price['id'] = pd.to_numeric(all_price.id)

In [ ]:
top_rest.isnull().sum()

In [ ]:
top_rest.groupby('id').id.nunique()

In [ ]:
top_rest[top_rest.city == 'Beirut'].price_level.isnull().sum()

In [ ]:
# Count and list the amount of null values
null_count = top_rest.price_level.isnull().groupby([top_rest['city']]).sum().astype(int).reset_index(name='count')
null_count

In [ ]:
null_count.sort_values('count')

In [ ]:
null_count[null_count['count'] > 45]

In [ ]:
# all 5 tables together
all_price.groupby('city')['price_level'].mean()

In [ ]:
# One table Best restaurant query
top_rest.groupby('city')['price_level'].mean()

In [ ]:
# 4 Tables price 1-4
price.groupby('city')['price_level'].mean()

In [ ]:
def graph_all(df):
    i = 1
    k = 5
    while k <= 140:
        sample = df[(df.id >= i) & (df.id <= k)]
        g = sns.FacetGrid(sample, col = "city")
        g.map(plt.hist, "price_level")
        plt.show()
        i += 5
        k += 5

In [ ]:
graph_all(all_food)

In [ ]:
graph_all(all_price)

In [ ]:
top_rest

In [ ]:
all_price

In [ ]:
plt.hist(all_price.price_level)

In [ ]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level

In [ ]:
food = to_city(all_price)
food

In [ ]:
food.reset_index(inplace=True)
food.drop(columns = ['avg_price', 'country', 'city'], inplace=True)


In [ ]:
food.set_index('id', inplace=True)

In [ ]:
food

In [ ]:
normalized = preprocessing.normalize(food)
normalized_food = pd.DataFrame(normalized)
normalized_food

In [ ]:
food.fillna(0, inplace=True)